<a href="https://colab.research.google.com/github/tiffanytang34/Age_Detection_Using_Facial_Image_with_CNN/blob/main/EDA_age_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports

from zipfile import ZipFile
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

Exploratory Data Analysis

### *facial-age* dataset (9,778 images)

In [12]:
# Unzipping the dataset file facial-age.zip

#!unzip facial-age.zip
facial_age_zip_path = "/content/drive/My Drive/Age_Detection/facial-age.zip"

with ZipFile(facial_age_zip_path, 'r') as myzip:
    myzip.extractall()
    print('Done unzipping facial-age.zip')

Done unzipping facial-age.zip


***facial-age* dataset** contains **9,778 RGB images of faces** in **PNG** format of size **200x200 pixels** each. The images are separated into folders where the **folder names correspond to the age labels of images inside those folders**.

In [13]:
# Accessing all folder names to get list of labels.

facial_age_path = "facial-age"
facial_age_folders = os.listdir(facial_age_path)

In [14]:
# Sorting the folder names (age labels) in ascending order.

facial_age_folders = np.sort(facial_age_folders)
facial_age_folders

array(['001', '002', '003', '004', '005', '006', '007', '008', '009',
       '010', '011', '012', '013', '014', '015', '016', '017', '018',
       '019', '020', '021', '022', '023', '024', '025', '026', '027',
       '028', '029', '030', '031', '032', '033', '034', '035', '036',
       '037', '038', '039', '040', '041', '042', '043', '044', '045',
       '046', '047', '048', '049', '050', '051', '052', '053', '054',
       '055', '056', '057', '058', '059', '060', '061', '062', '063',
       '064', '065', '066', '067', '068', '069', '070', '071', '072',
       '073', '074', '075', '076', '077', '078', '079', '080', '081',
       '082', '083', '084', '085', '086', '087', '088', '089', '090',
       '091', '092', '093', '095', '096', '099', '100', '101', '110'],
      dtype='<U3')